In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
2


In [4]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

def normalize_image(image):
    mean = np.array([0.435, 0.426, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # 이미지를 0과 1 사이의 값으로 스케일 조정
    normalized_image = image / 255.0
    
    # 평균을 빼고 표준 편차로 나누어 정규화
    normalized_image = (normalized_image - mean) / std
    
    return normalized_image


In [ ]:
def make_bumper_std(image, mask):
    h,w,_ = image.shape
    background = cv2.imread('./background.png')
    background = cv2.resize(background, (h,w))
    background_mask = background[:,:,0]

    target_image = cv2.imread('./data/224/target_val_image/svm_front_2021-06-04-14-37-37_00000000.png')
    target_mask = cv2.imread('./data/224/target_val_gt/svm_front_2021-06-04-14-37-37_00000000.png')

    target_image = cv2.resize(target_image, (h,w))
    target_mask = cv2.resize(target_mask, (h,w))

    bumper_image = image * background + target_image * (1-background)
    bumper_mask = mask * background_mask + target_mask * (1-background_mask)

    
    return bumper_image, bumper_mask

In [5]:
def make_bumper(image, mask):
    h,w,c = image.shape
    center = (w//2, int(h*0.375))  # x,y
    axis_length = (w//2, int(h*0.64))  # 장축 반지름과 단축 반지름

    # 타원 그리기 (타원을 1로 채우고 나머지 부분은 0으로 채움)
    oval = np.zeros((h,w,1),dtype=np.uint8)
    cv2.ellipse(oval, center, axis_length, 0, 0, 360, 1, -1)

    target_image_path = './data/224/train_target_image/TRAIN_TARGET_0001.png'
    target_image = cv2.imread(target_image_path, cv2.IMREAD_COLOR)
    target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)
    target_image = normalize_image(target_image)

    mixed_image = image * oval + target_image * (1-oval)
    mixed_mask = mask * oval[:,:,0] + np.ones_like(mask)*0*(1-oval[:,:,0])

    return mixed_image, mixed_mask



In [ ]:
import numpy as np
import cv2
import torch

def apply_fisheye_distortion(images, masks, label):
    # 이미지 크기 가져오기
    height, width, channel = images.shape

    # 카메라 매트릭스 생성
    focal_length = width / 2
    center_x = width / 2
    center_y = height / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)

    dist_num = label
    dist_coeffs = np.array([0, 0.05 * dist_num, 0, 0], dtype=np.float32)

    # 왜곡 보정
    undistorted_images = []
    undistorted_masks = []

    for i in range(batch):
        image = images[i].permute(1, 2, 0).cpu().numpy()  # 텐서를 NumPy 배열로 변환
        mask = masks[i].cpu().numpy()
        undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)
        undistorted_mask = cv2.undistort(mask, camera_matrix, dist_coeffs)
        undistorted_mask = np.round(undistorted_mask).astype(np.uint8)
        undistorted_mask[undistorted_mask > 12] = 12

        if label == 1:
            undistorted_image, undistorted_mask = make_bumper_std(undistorted_image, undistorted_mask)

        #다시 텐서로 변환
        undistorted_image = torch.from_numpy(undistorted_image).permute(2, 0, 1).float().to(device)
        undistorted_mask = torch.from_numpy(undistorted_mask).long().to(device)

        undistorted_images.append(undistorted_image)
        undistorted_masks.append(undistorted_mask)

    undistorted_images = torch.stack(undistorted_images, dim=0)
    undistorted_masks = torch.stack(undistorted_masks, dim=0)

    return undistorted_images, undistorted_masks


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        

        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주
        mask += 1
        mask[mask == 13] = 0
        

        
        image, mask = make_bumper_std(image, mask)


        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

     

transform = A.Compose(
    [   
        #A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        A.GaussNoise(var_limit=(10.0, 30.0), p=0.3),
        
        # 변형
        A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        #A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

## Model

In [7]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(self, x):
        return x.view_as(x)
    @staticmethod
    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return grad_output * (-1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 50)
        self.fc2 = nn.Linear(50, 3) # source = 0, target = 1 회귀 가정
        ##
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        #return torch.sigmoid(x)
        return x

class domain_linear(nn.Module):
    def __init__(self):
        super(domain_linear, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 1) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        return torch.sigmoid(x)
        #return x
class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 3x3 convolution
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU()
        
        # 3x3 convolution
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        
        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Adding the skip connection
        out += self.skip(identity)
        out = self.relu2(out)
        
        return out
#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv2,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv3,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        p = self.maxpool(x)
        
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv4,self).__init__()         
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,in_channels)
        self.identityblock4 = IdentityBlock(in_channels,in_channels)
        self.identityblock5 = IdentityBlock(in_channels,in_channels)
        self.identityblock6 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        x = self.identityblock5(x)
        x = self.identityblock6(x)
        p = self.maxpool(x)
        
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(Conv5,self).__init__() 
        self.identityblock1 = IdentityBlock(in_channels,in_channels)
        self.identityblock2 = IdentityBlock(in_channels,in_channels)
        self.identityblock3 = IdentityBlock(in_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        
    def forward(self,x):
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1) # output_padding 추가
        self.convblock1 = IdentityBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #print("x",x.shape,"skip: ",skip.shape)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Resnet34_Unet(nn.Module):
    def __init__(self,n_classes):
        super(Resnet34_Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv2 = Conv2(64,128)
        self.conv3 = Conv3(128,256)
        self.conv4 = Conv4(256,512)
        self.conv5 = Conv5(512,1024)
        
        self.middleconv = IdentityBlock(1024,2048)
        self.dropout = nn.Dropout2d(0.1) #
           
        self.decoder5 = DecoderBlock(1024)
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.transpose = nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1) # output_padding 추가
        
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        self.domain_classifier = domain_classifier()
        self.domain_linear = domain_linear()
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x0 = self.fbn1(x)
        x1 = self.frelu1(x)
        p = self.fmaxpooling(x1)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("conv1: ",x1.shape, "maxpooling: ",p.shape)
        x2,p = self.conv2(p)
        #print("conv2: ",x2.shape, "maxpooling: ",p.shape)
        x3,p = self.conv3(p)
        #print("conv3: ",x3.shape, "maxpooling: ",p.shape)
        x4,p = self.conv4(p)
        #print("conv4: ",x4.shape, "maxpooling: ",p.shape)
        x5,p = self.conv5(p)
        #print("conv5: ",x5.shape, "maxpooling: ",p.shape)
        
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #print("xm: ",xm.shape, "maxpooling: ",p.shape)
        xm = self.dropout(xm)
        
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        x = self.decoder3(x,x3) #14
        x = self.decoder2(x,x2)#28
        x = self.decoder1(x,x1)#55
        x = self.transpose(x)
        
        #print(x.shape)
        #x = F.interpolate(x, size=(224, 224))
        x_c = self.segmap(x)
        #x_d = self.domain_linear(x)
        #x_d = self.domain_classifier(x)
        
        
        return x_c

## Data Loader

In [8]:
LR = 0.001
EP = 10
BATCH_SIZE = 16
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
ALPHA = 0.00045
Label = [0]
N_LABELS = len(Label)
# model 초기화
#model = Resnet18_Unet(n_classes = N_CLASSES).to(device)
model = Resnet34_Unet(n_classes = N_CLASSES).to(device)
#model.load_state_dict(torch.load('./data/resnet34_1112_dropout_lr00045.pth'), strict=False)
#model = Resnet50_Unet(n_classes = N_CLASSES).to(device)
#model = Unet(n_classes = N_CLASSES).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 

loss_fn = nn.CrossEntropyLoss()

source_dataset = CustomDataset(csv_file='./data/896_csv/train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_target_dataset = CustomDataset(csv_file='./data/896_csv/val_source_CL.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)


## Training

In [9]:
import random
#torch.cuda.empty_cache()
import matplotlib.pyplot as plt
# import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="11_12_3d_res34_dropout",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

for epoch in range(EP):
    model.train()
    epoch_loss = 0
    seg_losses = 0
    domain_losses = 0
    # if epoch < 10:  # 전체 EP 40%
    #     alpha = 0.00045
    # elif epoch <25: # 전체 EP 30%
    #     alpha = 0.00045
    # else:          # 전체 EP 30%
    #     alpha = 0.00045
    train_class_ious = [[]]
    for source_images, source_masks in tqdm(source_dataloader,desc=f"Epoch: {epoch+1}"):
        random.shuffle(Label)
        for l in range(N_LABELS):
            label = Label[l]
            source_image, source_mask = apply_fisheye_distortion(source_images, source_masks, 1)
            source_image = source_image.float().to(device)
            source_mask = source_mask.long().to(device)
            source_outputs = model(source_image)
            
            optimizer.zero_grad()
            target_loss = loss_fn(source_outputs, source_mask)
            epoch_loss += target_loss.item()
            target_loss.backward()
            optimizer.step()
            # miou측정
            source_outputs = model(source_image)
            source_outputs = torch.softmax(source_outputs, dim=1).cpu()
            source_outputs = torch.argmax(source_outputs, dim=1).numpy()
            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
                train_class_ious[label].append(iou)
            #print(train_class_ious[0])
    
    for i in range(N_LABELS):
        buff = np.array(train_class_ious[i]).reshape(-1, N_CLASSES)
        buff = np.mean(buff, axis=0)
        print(f"\nLabel_{i}: IoU Scores Train") 
        for class_id, iou in enumerate(buff):
            print(f'Class{class_id:02d}: {iou:.4f}', end=" ")
            if (class_id+1) % 7 == 0:
                print()   
    print()    
    #print(f"Train seg Loss: {(seg_losses/(N_LABELS*len(source_dataloader)))}", f"Train dom Loss: {(domain_losses/(N_LABELS*len(source_dataloader)))}")
    print(f"Train Loss: {(epoch_loss/(N_LABELS*len(source_dataloader)))}")
    print(f"Train mIoU: {np.mean(train_class_ious)}" )
    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    val_epoch_loss = 0
    val_seg_loss = 0
    val_domain_loss = 0
    # valid
    with torch.no_grad():
        model.eval()

        for target_images, target_masks in tqdm(val_target_dataloader):
            target_images, target_masks = apply_fisheye_distortion(target_images, target_masks, 0)
            target_images = target_images.float().to(device)
            target_masks = target_masks.long().to(device)

            target_outputs = model(target_images)

            target_loss = loss_fn(target_outputs, target_masks)

            loss = target_loss

            val_epoch_loss += loss.item()

            # train 클래스별 IoU 계산
            target_outputs = torch.softmax(target_outputs, dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(-1, N_CLASSES)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=0)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id:02d}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    #print(f"Valid Seg Loss: {(val_seg_loss/len(val_target_dataloader))}",f"Valid dom Loss: {(val_domain_loss/len(val_target_dataloader))}")
    print(f"Valid Loss: {(val_epoch_loss/len(val_target_dataloader))}")
    print(f"Valid mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")


#     # log metrics to wandb
#     wandb.log({"train score": np.mean(train_class_ious)})
#     wandb.log({"val score": fish_val_mIoU})
#     wandb.log({"train loss": (epoch_loss/(N_LABELS*len(source_dataloader)))})
#     wandb.log({"val loss": (val_epoch_loss/len(val_target_dataloader))})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()



Epoch: 1:   0%|          | 0/138 [00:00<?, ?it/s]

Epoch: 1: 100%|██████████| 138/138 [02:08<00:00,  1.08it/s]



Label_0: IoU Scores Train
Class00: 0.4275 Class01: 0.4971 Class02: 0.0000 Class03: 0.3326 Class04: 0.0000 Class05: 0.0000 Class06: 0.0000 
Class07: 0.0000 Class08: 0.3486 Class09: 0.6161 Class10: 0.0000 Class11: 0.0000 Class12: 0.1563 
Train Loss: 0.9118620913097824
Train mIoU: 0.18294083318724547


100%|██████████| 61/61 [00:49<00:00,  1.23it/s]



--IoU Scores Fish val--
Class00: 0.0178 Class01: 0.0200 Class02: 0.0000 Class03: 0.0074 Class04: 0.0000 Class05: 0.0000 Class06: 0.0000 
Class07: 0.0000 Class08: 0.0012 Class09: 0.0147 Class10: 0.0000 Class11: 0.0000 Class12: 0.0002 
Epoch1
Valid Loss: 6.442216083651683
Valid mIoU: 0.004713820277403963
___________________________________________________________________________________________



...


--IoU Scores Fish val--
Class00: 0.0058 Class01: 0.0060 Class02: 0.0001 Class03: 0.0087 Class04: 0.0013 Class05: 0.0000 Class06: 0.0041 
Class07: 0.0017 Class08: 0.0017 Class09: 0.0150 Class10: 0.0000 Class11: 0.0000 Class12: 0.0001 
Epoch9
Valid Loss: 9.52391827692751
Valid mIoU: 0.0034321048734552687
___________________________________________________________________________________________



Epoch: 10: 100%|██████████| 138/138 [01:55<00:00,  1.20it/s]



Label_0: IoU Scores Train
Class00: 0.4660 Class01: 0.6664 Class02: 0.0991 Class03: 0.4877 Class04: 0.2543 Class05: 0.0000 Class06: 0.0946 
Class07: 0.1713 Class08: 0.5218 Class09: 0.7236 Class10: 0.0000 Class11: 0.0000 Class12: 0.4887 
Train Loss: 0.4880532982988634
Train mIoU: 0.30566323474188845


 70%|███████   | 43/61 [00:21<00:08,  2.01it/s]


KeyboardInterrupt: 

In [ ]:
import random
#torch.cuda.empty_cache()
import matplotlib.pyplot as plt
# import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="11_12_3d_res34_dropout",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

model = Resnet34_Unet(n_classes = N_CLASSES).to(device)

for epoch in range(EP):
    model.train()
    epoch_loss = 0
    seg_losses = 0
    domain_losses = 0
    # if epoch < 10:  # 전체 EP 40%
    #     alpha = 0.00045
    # elif epoch <25: # 전체 EP 30%
    #     alpha = 0.00045
    # else:          # 전체 EP 30%
    #     alpha = 0.00045
    train_class_ious = [[]]
    for source_images, source_masks in tqdm(source_dataloader,desc=f"Epoch: {epoch+1}"):
        random.shuffle(Label)
        for l in range(N_LABELS):
            label = Label[l]
            source_image, source_mask = apply_fisheye_distortion(source_images, source_masks, 0)
            source_image = source_image.float().to(device)
            source_mask = source_mask.long().to(device)
            source_outputs = model(source_image)
            
            optimizer.zero_grad()
            target_loss = loss_fn(source_outputs, source_mask)
            epoch_loss += target_loss.item()
            target_loss.backward()
            optimizer.step()
            # miou측정
            source_outputs = model(source_image)
            source_outputs = torch.softmax(source_outputs, dim=1).cpu()
            source_outputs = torch.argmax(source_outputs, dim=1).numpy()
            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
                train_class_ious[label].append(iou)
            #print(train_class_ious[0])
    
    for i in range(N_LABELS):
        buff = np.array(train_class_ious[i]).reshape(-1, N_CLASSES)
        buff = np.mean(buff, axis=0)
        print(f"\nLabel_{i}: IoU Scores Train") 
        for class_id, iou in enumerate(buff):
            print(f'Class{class_id:02d}: {iou:.4f}', end=" ")
            if (class_id+1) % 7 == 0:
                print()   
    print()    
    print(f"Train seg Loss: {(seg_losses/(N_LABELS*len(source_dataloader)))}", f"Train dom Loss: {(domain_losses/(N_LABELS*len(source_dataloader)))}")
    print(f"Train Loss: {(epoch_loss/(N_LABELS*len(source_dataloader)))}")
    print(f"Train mIoU: {np.mean(train_class_ious)}" )
    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    val_epoch_loss = 0
    val_seg_loss = 0
    val_domain_loss = 0
    # valid
    with torch.no_grad():
        model.eval()

        for target_images, target_masks in tqdm(val_target_dataloader):
            target_images, target_masks = apply_fisheye_distortion(target_images, target_masks, 0)
            target_images = target_images.float().to(device)
            target_masks = target_masks.long().to(device)

            target_outputs = model(target_images)

            target_loss = loss_fn(target_outputs, target_masks)

            loss = target_loss

            val_epoch_loss += loss.item()

            # train 클래스별 IoU 계산
            target_outputs = torch.softmax(target_outputs, dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(-1, N_CLASSES)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=0)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id:02d}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Valid Seg Loss: {(val_seg_loss/len(val_target_dataloader))}",f"Valid dom Loss: {(val_domain_loss/len(val_target_dataloader))}")
    print(f"Valid Loss: {(val_epoch_loss/len(val_target_dataloader))}")
    print(f"Valid mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")


#     # log metrics to wandb
#     wandb.log({"train score": np.mean(train_class_ious)})
#     wandb.log({"val score": fish_val_mIoU})
#     wandb.log({"train loss": (epoch_loss/(N_LABELS*len(source_dataloader)))})
#     wandb.log({"val loss": (val_epoch_loss/len(val_target_dataloader))})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()



Epoch: 1: 100%|██████████| 138/138 [01:31<00:00,  1.52it/s]



Label_0: IoU Scores Train
Class00: 0.0925 Class01: 0.0024 Class02: 0.0068 Class03: 0.1132 Class04: 0.0090 Class05: 0.0069 Class06: 0.0033 
Class07: 0.0088 Class08: 0.0101 Class09: 0.1779 Class10: 0.0018 Class11: 0.0008 Class12: 0.0431 
Train seg Loss: 0.0 Train dom Loss: 0.0
Train Loss: 2.5113008195075435
Train mIoU: 0.03666853670035566


100%|██████████| 61/61 [00:21<00:00,  2.88it/s]



--IoU Scores Fish val--
Class00: 0.0240 Class01: 0.0121 Class02: 0.0369 Class03: 0.0022 Class04: 0.0332 Class05: 0.0135 Class06: 0.0013 
Class07: 0.0009 Class08: 0.0009 Class09: 0.0306 Class10: 0.0186 Class11: 0.0003 Class12: 0.0001 
Epoch1
Valid Seg Loss: 0.0 Valid dom Loss: 0.0
Valid Loss: 2.69889332036503
Valid mIoU: 0.01342925382697639
___________________________________________________________________________________________



Epoch: 2: 100%|██████████| 138/138 [01:35<00:00,  1.44it/s]



Label_0: IoU Scores Train
Class00: 0.0925 Class01: 0.0024 Class02: 0.0068 Class03: 0.1138 Class04: 0.0090 Class05: 0.0068 Class06: 0.0033 
Class07: 0.0087 Class08: 0.0101 Class09: 0.1775 Class10: 0.0018 Class11: 0.0008 Class12: 0.0429 
Train seg Loss: 0.0 Train dom Loss: 0.0
Train Loss: 2.5114863558091978
Train mIoU: 0.036649199360629534


100%|██████████| 61/61 [00:24<00:00,  2.49it/s]



--IoU Scores Fish val--
Class00: 0.0243 Class01: 0.0125 Class02: 0.0392 Class03: 0.0022 Class04: 0.0334 Class05: 0.0134 Class06: 0.0013 
Class07: 0.0009 Class08: 0.0009 Class09: 0.0311 Class10: 0.0199 Class11: 0.0003 Class12: 0.0001 
Epoch2
Valid Seg Loss: 0.0 Valid dom Loss: 0.0
Valid Loss: 2.711941953565254
Valid mIoU: 0.013794959531694526
___________________________________________________________________________________________



Epoch: 3:  11%|█         | 15/138 [00:10<01:21,  1.52it/s]